In [1]:
import pandas as pd
import numpy as np
import requests
import zipfile
import os
from io import BytesIO, StringIO
from sklearn.model_selection import train_test_split
# import kagglehub
import random
from sklearn.utils import resample

In [2]:
# Setting random seed to replicate results
DEFAULT_RANDOM_SEED = 2021

def seedBasic(seed=DEFAULT_RANDOM_SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

def seedEverything(seed=DEFAULT_RANDOM_SEED):
    seedBasic(seed)

seedEverything(2021)

In [3]:
# # Download latest version
# path = kagglehub.dataset_download("chethuhn/network-intrusion-dataset")

# print("Path to dataset files:", path)

100%|██████████| 230M/230M [00:03<00:00, 64.8MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/chethuhn/network-intrusion-dataset/versions/1


In [4]:
# def load_and_preprocess_data(path):
#     all_files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith(".csv")]
#     df_list = [pd.read_csv(f) for f in all_files]
#     df = pd.concat(df_list, ignore_index=True)

#     df.columns = df.columns.str.strip().str.lower()

#     # Remove infinite values
#     df.replace([np.inf, -np.inf], np.nan, inplace=True)

#     # Drop rows with missing values and duplicates
#     df.dropna(inplace=True)
#     df.drop_duplicates(inplace=True)

#     del df_list, all_files

#     return df

In [5]:
# df = load_and_preprocess_data(path)

<h2> Load the dataset and preproces </h2>

In [3]:
data1 = pd.read_csv('/Users/akash/archive/Monday-WorkingHours.pcap_ISCX.csv')
data2 = pd.read_csv('/Users/akash/archive/Tuesday-WorkingHours.pcap_ISCX.csv')
data3 = pd.read_csv('/Users/akash/archive/Wednesday-workingHours.pcap_ISCX.csv')
data4 = pd.read_csv('/Users/akash/archive/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv')
data5 = pd.read_csv('/Users/akash/archive/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv')
data6 = pd.read_csv('/Users/akash/archive/Friday-WorkingHours-Morning.pcap_ISCX.csv')
data7 = pd.read_csv('/Users/akash/archive/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv')
data8 = pd.read_csv('/Users/akash/archive/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')

In [4]:
data_list = [data1, data2, data3, data4, data5, data6, data7, data8]
df = pd.concat(data_list)

In [5]:
#Remove prefix spaces
df.columns = df.columns.str.lstrip()

<h2>Removing duplicates, Removing missing values </h2>

In [6]:
df.replace([np.inf, -np.inf], np.nan, inplace=True) #Removing infinite values

In [7]:
df.drop_duplicates(keep=False,inplace=True) #remove duplicate

In [8]:
df.dropna(inplace=True) # remove rows containing missing values

In [6]:
# # Free up RAM
# import gc
# gc.collect()

9

In [20]:
subset_percentage = 0.05 #change the subset percentage as per the requirement, 10% and 5%
min_sample_size = 1000
subset_size = max(int(len(df) * subset_percentage), min_sample_size)

subset_list = []
for class_name, class_data in df.groupby('Label'):
    class_proportion = len(class_data) / len(df)
    class_sample_size = min(max(int(subset_size * class_proportion), min_sample_size), len(class_data))
    sampled_data = resample(class_data, n_samples=class_sample_size, random_state=DEFAULT_RANDOM_SEED, replace=False)
    subset_list.append(sampled_data)

subset_df = pd.concat(subset_list).sample(frac=1, random_state=DEFAULT_RANDOM_SEED).reset_index(drop=True)

In [21]:
# Perform train-test split
train_df, test_df = train_test_split(subset_df, test_size=0.33,random_state=DEFAULT_RANDOM_SEED)

# Display dataset shapes
print("Train Dataset Shape:", train_df.shape)
print("Test Dataset Shape:", test_df.shape)

# Show first few rows of training data
train_df.head()

Train Dataset Shape: (85313, 79)
Test Dataset Shape: (42021, 79)


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
31702,53,102919,4,4,164,408,41,41,41.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
22819,53,30727,4,2,136,182,34,34,34.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
114879,53,31080,2,2,84,204,42,42,42.0,0.00000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
85375,53,108170,1,1,48,190,48,48,48.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
65838,80,85434190,6,6,330,11595,312,0,55.0,125.93808,...,20,11000.0,0.0,11000,11000,85200000.0,0.0,85200000,85200000,DoS Hulk


<h2> Create train set for each subset </h2>

In [22]:
url = "data.csv"
syn_df = pd.read_csv(url)
syn_df.drop('Unnamed: 0',axis=1,inplace=True)
syn_df.columns = syn_df.columns.str.strip().str.lower()

syn_df.head()

,destination port,flow duration,total fwd packets,total backward packets,total length of fwd packets,total length of bwd packets,fwd packet length max,fwd packet length min,fwd packet length mean,fwd packet length std,...,min_seg_size_forward,active mean,active std,active max,active min,idle mean,idle std,idle max,idle min,label
0,50,59355,12,4,41,0,740,0,4.042365,0.790155,...,-282,0.000000,0.0,0,513,6.674454e+04,0.000000,0,28865,SSH-Patator
1,52,30580407,7,5,452,6605,392,0,123.025224,171.642192,...,-702,1201.570230,0.0,0,1652,1.062313e+07,4498.683927,20514533,5997158,DoS GoldenEye
2,112,102992405,2,2,35,0,18,6,6.493130,0.000000,...,-745,659.316284,0.0,0,1311,1.012151e+08,6542.295628,98763141,100435419,DoS slowloris
3,453,60682560,1,13,53,806,32,1,37.333180,13.381788,...,-409,101479.907127,0.0,23399,44,6.932652e+07,2442.130102,58719465,75697953,BENIGN
4,29,6332515,13,11,92,1693,29,1,7.817241,7.635458,...,-451,0.000000,0.0,1364,207,7.143109e+04,0.000000,0,39215,FTP-Patator


<h2>Making sure all columns match </h2>

In [23]:
train_df.columns = train_df.columns.str.strip().str.lower()
syn_df.columns = syn_df.columns.str.strip().str.lower()
test_df.columns = test_df.columns.str.strip().str.lower()

# Verify columns match
print("Train columns:", train_df.columns.tolist())
print("Syn columns:", syn_df.columns.tolist())
print("Test columns:", test_df.columns.tolist())

Train columns: ['destination port', 'flow duration', 'total fwd packets', 'total backward packets', 'total length of fwd packets', 'total length of bwd packets', 'fwd packet length max', 'fwd packet length min', 'fwd packet length mean', 'fwd packet length std', 'bwd packet length max', 'bwd packet length min', 'bwd packet length mean', 'bwd packet length std', 'flow bytes/s', 'flow packets/s', 'flow iat mean', 'flow iat std', 'flow iat max', 'flow iat min', 'fwd iat total', 'fwd iat mean', 'fwd iat std', 'fwd iat max', 'fwd iat min', 'bwd iat total', 'bwd iat mean', 'bwd iat std', 'bwd iat max', 'bwd iat min', 'fwd psh flags', 'bwd psh flags', 'fwd urg flags', 'bwd urg flags', 'fwd header length', 'bwd header length', 'fwd packets/s', 'bwd packets/s', 'min packet length', 'max packet length', 'packet length mean', 'packet length std', 'packet length variance', 'fin flag count', 'syn flag count', 'rst flag count', 'psh flag count', 'ack flag count', 'urg flag count', 'cwe flag count', 

In [24]:
assert set(train_df.columns) == set(syn_df.columns), "Columns don't match!"

In [25]:
combined_train_df = pd.concat([train_df, syn_df], ignore_index=True)
print("Updated Combined Train Dataset Shape:", combined_train_df.shape)

Updated Combined Train Dataset Shape: (95313, 79)


In [26]:
# github_url = "https://github.com/akashvenus/Final_Project/raw/main/data.csv"
# response = requests.get(github_url)

# github_df = pd.read_csv(StringIO(response.text))
# github_df.drop('Unnamed: 0',axis=1,inplace=True)
# github_df.columns = github_df.columns.str.strip().str.lower()


# # Combine with test_df
# combined_test_df = pd.concat([test_df, github_df], ignore_index=True)
# print("Updated Combined Test Dataset Shape:", combined_test_df.shape)

Updated Combined Test Dataset Shape: (94615, 79)


In [26]:
test_df.head(5)

,destination port,flow duration,total fwd packets,total backward packets,total length of fwd packets,total length of bwd packets,fwd packet length max,fwd packet length min,fwd packet length mean,fwd packet length std,...,min_seg_size_forward,active mean,active std,active max,active min,idle mean,idle std,idle max,idle min,label
17800,53,30449,1,1,49,93,49,49,49.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
26720,443,297054,15,14,851,14141,373,0,56.733333,113.256010,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
54825,53,75095647,2,2,115,242,58,57,57.500000,0.707107,...,32,50660.0,0.0,50660,50660,74900000.0,0.0,74900000,74900000,BENIGN
104490,53,209,2,2,90,172,45,45,45.000000,0.000000,...,44,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
53988,443,30184469,12,6,859,3157,373,0,71.583333,123.682481,...,32,167703.0,0.0,167703,167703,30000000.0,0.0,30000000,30000000,BENIGN


In [29]:
# github_df.head(5)

,destination port,flow duration,total fwd packets,total backward packets,total length of fwd packets,total length of bwd packets,fwd packet length max,fwd packet length min,fwd packet length mean,fwd packet length std,...,min_seg_size_forward,active mean,active std,active max,active min,idle mean,idle std,idle max,idle min,label
0,50,59355,12,4,41,0,740,0,4.042365,0.790155,...,-282,0.000000,0.0,0,513,6.674454e+04,0.000000,0,28865,SSH-Patator
1,52,30580407,7,5,452,6605,392,0,123.025224,171.642192,...,-702,1201.570230,0.0,0,1652,1.062313e+07,4498.683927,20514533,5997158,DoS GoldenEye
2,112,102992405,2,2,35,0,18,6,6.493130,0.000000,...,-745,659.316284,0.0,0,1311,1.012151e+08,6542.295628,98763141,100435419,DoS slowloris
3,453,60682560,1,13,53,806,32,1,37.333180,13.381788,...,-409,101479.907127,0.0,23399,44,6.932652e+07,2442.130102,58719465,75697953,BENIGN
4,29,6332515,13,11,92,1693,29,1,7.817241,7.635458,...,-451,0.000000,0.0,1364,207,7.143109e+04,0.000000,0,39215,FTP-Patator


In [27]:
combined_train_df.head(5)

,destination port,flow duration,total fwd packets,total backward packets,total length of fwd packets,total length of bwd packets,fwd packet length max,fwd packet length min,fwd packet length mean,fwd packet length std,...,min_seg_size_forward,active mean,active std,active max,active min,idle mean,idle std,idle max,idle min,label
0,53,102919,4,4,164,408,41,41,41.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,53,30727,4,2,136,182,34,34,34.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,53,31080,2,2,84,204,42,42,42.0,0.00000,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,53,108170,1,1,48,190,48,48,48.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,80,85434190,6,6,330,11595,312,0,55.0,125.93808,...,20,11000.0,0.0,11000,11000,85200000.0,0.0,85200000,85200000,DoS Hulk


In [28]:
train_df.to_csv("train.csv")

In [29]:
test_df.to_csv("test.csv")

In [30]:
combined_train_df.to_csv("combined_train_df.csv")